# **IMPORT LIBRARIES**

In [1]:
pip install pyvi

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install underthesea

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [119]:
from underthesea import word_tokenize, pos_tag
from pyvi import ViTokenizer, ViPosTagger
from collections import Counter

# Sklearn: Các mô-đun hỗ trợ Machine Learning
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import make_classification

# thư viện NLP tiếng Việt
from tqdm import tqdm
import pickle
import unidecode
import gensim

# **IMPORT DATA**

In [8]:
#url = 'https://raw.githubusercontent.com/anhlam-road/Med_Specialty_Classifier/refs/heads/main/disease_11k.csv'
#url = 'https://raw.githubusercontent.com/anhlam-road/Med_Specialty_Classifier/refs/heads/main/train_d.csv'
url = 'https://raw.githubusercontent.com/anhlam-road/Med_Specialty_Classifier/refs/heads/main/raw_disease_updated.csv'
df = pd.read_csv(url)
df

,Disease,Department,Symptom,Category
0,Ảo giác,Tâm thần,"Người bệnh có thể bị căng thẳng, mất ngủ, đờ đ...",Knownledge
1,Rối loạn đa nhân cách,Tâm thần,Người mắc bệnh rối loạn đa nhân cách thường có...,Knownledge
2,Rối loạn giấc ngủ,Tâm thần,Triệu chứng chung của những người bị mắc chứng...,Knownledge
3,Trầm cảm,Tâm thần,Bệnh nhân khi bị trầm cảm sẽ có các biểu hiện ...,Knownledge
4,Mất ngủ ở người cao tuổi,Tâm thần,Triệu chứng của mất ngủ ở người cao tuổi là kh...,Knownledge
...,...,...,...,...
28022,NaN,Dị ứng,"Tôi hay bị mẩn ngứa khi ăn cua, ghẹ (đặc biệt ...",News
28023,NaN,Hô hấp,Dấu hiệu của bệnh viêm phế quản là gì? .Viêm p...,News
28024,NaN,Nam khoa,Vợ chồng cháu đi khám hiếm muộn thì cần phải l...,News
28025,NaN,Nội tiết,Bệnh nhân tiền sử tiểu đường tuýp 2 nóng rát t...,News


In [10]:
# Thông tin về số dòng, số cột, kiểu dữ liệu
print("Thông tin tổng quan về DataFrame:")
print(df.info())

# Kiểm tra missing values
print("\nSố lượng giá trị bị thiếu trong mỗi cột:")
print(df.isnull().sum())

# Kiểm tra số lượng dòng trùng
duplicate_rows = df.duplicated().sum()
print(f"\nSố lượng dòng trùng lặp: {duplicate_rows}")

Thông tin tổng quan về DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28027 entries, 0 to 28026
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Disease     5029 non-null   object
 1   Department  28027 non-null  object
 2   Symptom     28027 non-null  object
 3   Category    28027 non-null  object
dtypes: object(4)
memory usage: 876.0+ KB
None

Số lượng giá trị bị thiếu trong mỗi cột:
Disease       22998
Department        0
Symptom           0
Category          0
dtype: int64

Số lượng dòng trùng lặp: 762


# **CLEAN SYMPTOM**

In [ ]:
#CÁC HÀM XỬ LÝ NỘI DUNG SYMPTOM

# **PRE - PROCESSING**

In [18]:
# Xóa dòng chứa giá trị bị thiếu
#df1 = df.dropna()

df1 = df.drop('Disease', axis=1)

# Xóa dòng trùng lặp
df1 = df1.drop_duplicates()

# Kiểm tra missing values
print("\nSố lượng giá trị bị thiếu trong mỗi cột:")
print(df1.isnull().sum())

# Kiểm tra số lượng dòng trùng
duplicate_rows = df1.duplicated().sum()
print(f"\nSố lượng dòng trùng lặp: {duplicate_rows}")

print(df1.info())


Số lượng giá trị bị thiếu trong mỗi cột:
Department    0
Symptom       0
Category      0
dtype: int64

Số lượng dòng trùng lặp: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 27257 entries, 0 to 28026
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Department  27257 non-null  object
 1   Symptom     27257 non-null  object
 2   Category    27257 non-null  object
dtypes: object(3)
memory usage: 851.8+ KB
None


## DEPARTMENT

In [29]:
df2 = df1.copy()
print(df2["Department"].value_counts())

Xét nghiệm              2740
Nội tiết                2033
Truyền nhiễm            1871
Tai mũi họng            1668
Dinh dưỡng              1499
Răng hàm mặt            1485
Da liễu                 1384
Hô hấp                  1373
Tiêu hóa                1186
Thần kinh               1101
Máu                     1007
Dị ứng                   992
Nội thần kinh            987
Mắt                      897
Tiết niệu                809
Tim mạch                 703
Ung thư                  644
Ngoại thần kinh          631
Tâm lý                   609
Tâm thần                 515
Mạch máu                 383
Ung bướu                 359
Cơ xương khớp            347
Tiêu hóa - Gan mật       347
Nam khoa                 302
Ngoại lồng ngực          277
Thận tiết niệu           135
Sức khỏe sinh sản        133
Da tóc móng              122
Di truyền                114
Nội tiết chuyển hóa       93
Sản phụ khoa              92
Sức khỏe giới tính        74
Nam học                   35
Nuôi dạy con  

In [90]:
# Tạo từ điển mapping cho các nhãn tương đồng và chuẩn hóa về chữ thường không dấu
department_mapping = {
    "Tim mach Long nguc": ["tim mach", "ngoai long nguc"],
    "Tieu hoa": ["tieu hoa", "tieu hoa - gan mat"],
    "Co Xuong Khop": ["co xuong khop"],
    "Than Tiet nieu": ["tiet nieu", "than tiet nieu"],
    "Noi tiet": ["noi tiet", "noi tiet chuyen hoa"],
    "Di ung": ["di ung"],
    "Truyen nhiem": ["truyen nhiem"],
    "Da lieu": ["da lieu", "da toc mong"],
    "Than kinh": ["noi than kinh", "ngoai than kinh", "than kinh", "dau mat co"],
    "Tam than": ["tam than", "tam ly"],
    "Ung buou": ["ung thu", "ung buou"],
    "Suc khoe gioi tinh": ["suc khoe gioi tinh", "suc khoe nam gioi", "suc khoe nu gioi", "suc khoe tinh duc", "nam hoc", "nam khoa"],
    "Phu san": ["suc khoe sinh san", "san phu khoa", "vu nhu", "mang thai", "ho tro sinh san ivf", "nuoi day con"],
    "Tai Mui Hong": ["tai mui hong"],
    "Rang Ham Mat": ["rang ham mat"],
    "Mat": ["mat", "nhan khoa"],
    "Mau": ["mau", "mach mau", "mau mien dich"],
    "Xet nghiem": ["xet nghiem", "di truyen"],
    "Dinh duong": ["dinh duong", "dinh duong tiet che", "bi quyet song khoe", "giac ngu ngon"],
    "Ho hap": ["ho hap"],
}
#Xoá chan doan hinh anh, nhi, suc khoe tong quat
# Hàm chuyển đổi chuỗi thành không dấu
def remove_accents(input_str):
    return unidecode.unidecode(input_str)

# Hàm chuẩn hóa theo từ điển mapping (không dấu)
def map_department(department):
    department = department.strip().lower()  # Chuyển về chữ thường và loại bỏ khoảng trắng thừa
    department = remove_accents(department)  # Loại bỏ dấu
    for key, values in department_mapping.items():
        # Kiểm tra nếu department có trong danh sách các giá trị đã chuẩn hóa không dấu
        if department in [remove_accents(val).lower() for val in values]:
            return key
    return department  # Nếu không có trong mapping thì giữ nguyên

# Danh sách các department cần xoá (đã chuẩn hoá không dấu và chữ thường)
departments_to_remove = ["chan doan hinh anh", "nhi", "suc khoe tong quat"]


In [92]:
df2 = df1.copy()

# Áp dụng mapping vào cột Department
df2["Department"] = df2["Department"].apply(map_department)

# Xoá các dòng có Department nằm trong danh sách trên
df2 = df2[~df2["Department"].isin(departments_to_remove)]

# Kiểm tra lại các giá trị duy nhất sau khi mapping
print("Các giá trị duy nhất sau khi mapping:", df2["Department"].unique())

# Xem kết quả
df2.head()

Các giá trị duy nhất sau khi mapping: ['Tam than' 'Than Tiet nieu' 'Ho hap' 'Than kinh' 'Truyen nhiem'
 'Tim mach Long nguc' 'Suc khoe gioi tinh' 'Noi tiet' 'Tieu hoa' 'Mau'
 'Ung buou' 'Xet nghiem' 'Rang Ham Mat' 'Tai Mui Hong' 'Di ung' 'Mat'
 'Da lieu' 'Dinh duong' 'Phu san' 'Co Xuong Khop']


,Department,Symptom,Category
0,Tam than,"Người bệnh có thể bị căng thẳng, mất ngủ, đờ đ...",Knownledge
1,Tam than,Người mắc bệnh rối loạn đa nhân cách thường có...,Knownledge
2,Tam than,Triệu chứng chung của những người bị mắc chứng...,Knownledge
3,Tam than,Bệnh nhân khi bị trầm cảm sẽ có các biểu hiện ...,Knownledge
4,Tam than,Triệu chứng của mất ngủ ở người cao tuổi là kh...,Knownledge


In [94]:
print(df2["Department"].value_counts())

Xet nghiem            2854
Than kinh             2729
Noi tiet              2126
Truyen nhiem          1871
Tai Mui Hong          1668
Tieu hoa              1533
Dinh duong            1509
Da lieu               1506
Rang Ham Mat          1485
Mau                   1421
Ho hap                1373
Tam than              1124
Ung buou              1003
Di ung                 992
Tim mach Long nguc     980
Than Tiet nieu         944
Mat                    917
Suc khoe gioi tinh     468
Co Xuong Khop          347
Phu san                322
Name: Department, dtype: int64


In [96]:
pivot_table = df1.pivot_table(index="Department", columns="Category", aggfunc='size', fill_value=0)
print(pivot_table)

Category              Knownledge  News    QA
Department                                  
Bí quyết sống khỏe             7     0     0
Chẩn đoán hình ảnh             3     0    25
Cơ xương khớp                320     0    27
Da liễu                       95  1047   242
Da tóc móng                  122     0     0
Di truyền                      9   105     0
Dinh dưỡng                     2  1008   489
Dinh dưỡng tiết chế            1     0     0
Dị ứng                        29   733   230
Giấc ngủ ngon                  2     0     0
Hô hấp                       169  1032   172
Hỗ trợ sinh sản ivf           14     0     0
Mang thai                     20     0     0
Máu                           48   399   560
Máu miễn dịch                 31     0     0
Mạch máu                      41   342     0
Mắt                          124   371   402
Nam học                       35     0     0
Nam khoa                       5   297     0
Ngoại lồng ngực               11   266     0
Ngoại thần

In [98]:
pivot_table = df2.pivot_table(index="Department", columns="Category", aggfunc='size', fill_value=0)
print(pivot_table)

Category            Knownledge  News    QA
Department                                
Co Xuong Khop              320     0    27
Da lieu                    217  1047   242
Di ung                      29   733   230
Dinh duong                  12  1008   489
Ho hap                     169  1032   172
Mat                        137   371   409
Mau                        120   741   560
Noi tiet                   181  1712   233
Phu san                    276     0    46
Rang Ham Mat               107   984   394
Suc khoe gioi tinh         168   297     3
Tai Mui Hong               214   796   658
Tam than                   155   662   307
Than Tiet nieu             184   586   174
Than kinh                  424  1589   716
Tieu hoa                   452   781   300
Tim mach Long nguc         504   410    66
Truyen nhiem               253  1586    32
Ung buou                   292   591   120
Xet nghiem                  16  1554  1284


## SYMPTOM

In [64]:
# Hàm xử lý dữ liệu trong cột Symptom
def preprocess_symptom(symptom):
    symptom = gensim.utils.simple_preprocess(symptom)  # Tiền xử lý văn bản
    symptom = ' '.join(symptom)  # Chuyển danh sách từ thành chuỗi
    symptom = ViTokenizer.tokenize(symptom)  # Tách từ tiếng Việt
    return symptom

In [62]:
df3 = df2.copy()
# Áp dụng xử lý lên cột Symptom
df3["Symptom"] = df3["Symptom"].apply(preprocess_symptom)

# Xem kết quả sau khi xử lý
df3.head()

,Department,Symptom,Category
0,Tam than,người_bệnh có_thể bị căng_thẳng mất_ngủ đờ_đẫn...,Knownledge
1,Tam than,người mắc bệnh rối_loạn đa nhân_cách thường có...,Knownledge
2,Tam than,triệu_chứng chung của những người bị mắc chứng...,Knownledge
3,Tam than,bệnh_nhân khi bị trầm_cảm sẽ có các biểu_hiện ...,Knownledge
4,Tam than,triệu_chứng của mất_ngủ người cao_tuổi là khó ...,Knownledge


## LABEL ENCONDING

In [100]:
encoder = preprocessing.LabelEncoder()
df3["Department_Encoded"] = encoder.fit_transform(df3["Department"])

encoder.classes_

array(['Co Xuong Khop', 'Da lieu', 'Di ung', 'Dinh dưỡng', 'Hô hấp',
       'Mat', 'Mau', 'Noi tiet', 'Phu san', 'Rang Ham Mat',
       'Suc khoe gioi tinh', 'Tai Mui Hong', 'Tam than', 'Than Tiet nieu',
       'Than kinh', 'Tieu hoa', 'Tim mach Long nguc', 'Truyen nhiem',
       'Ung buou', 'Xet nghiem'], dtype=object)

# **SPLIT TRAIN - TEST**

In [102]:
df4 = df3.copy() 

# Đếm số dòng theo cột 'Category'
category_counts = df4['Category'].value_counts()

# In kết quả
print(category_counts)

News          16480
QA             6462
Knownledge     4230
Name: Category, dtype: int64


In [104]:
df4 = df4.drop('Category', axis=1)
df4.head()

,Department,Symptom,Department_Encoded
0,Tam than,người_bệnh có_thể bị căng_thẳng mất_ngủ đờ_đẫn...,12
1,Tam than,người mắc bệnh rối_loạn đa nhân_cách thường có...,12
2,Tam than,triệu_chứng chung của những người bị mắc chứng...,12
3,Tam than,bệnh_nhân khi bị trầm_cảm sẽ có các biểu_hiện ...,12
4,Tam than,triệu_chứng của mất_ngủ người cao_tuổi là khó ...,12


In [110]:
X = df4['Symptom']
y = df4['Department_Encoded']

In [112]:
# Chia tập train/test có giữ tỷ lệ label (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Kiểm tra phân phối label
print("Phân phối label trong tập train:")
print(pd.Series(y_train).value_counts().sort_index())

print("\nPhân phối label trong tập test:")
print(pd.Series(y_test).value_counts().sort_index())

Phân phối label trong tập train:
0      278
1     1205
2      794
3     1207
4     1098
5      734
6     1137
7     1701
8      258
9     1188
10     374
11    1334
12     899
13     755
14    2183
15    1226
16     784
17    1497
18     802
19    2283
Name: Department_Encoded, dtype: int64

Phân phối label trong tập test:
0      69
1     301
2     198
3     302
4     275
5     183
6     284
7     425
8      64
9     297
10     94
11    334
12    225
13    189
14    546
15    307
16    196
17    374
18    201
19    571
Name: Department_Encoded, dtype: int64


In [116]:
#y_test.head()

24264    16
4248      2
18883    12
17467     7
8185     15
Name: Department_Encoded, dtype: int32

# **TF - IDF VECTORS: TRANSFORM BY SVD TO DECREASE NUMBER OF DIMENSIONS**
*   WORD LEVEL
*   NGRAM LEVEL
*   NGRAM CHAR LEVEL

In [125]:
# Gộp toàn bộ văn bản thành 1 chuỗi lớn
all_text = " ".join(df4['Symptom'])

# Tách từ và tạo tập hợp từ duy nhất
unique_words = set(all_text.split())

In [127]:
# Kết quả
print(f"Tổng số từ duy nhất trong X_data đã qua xử lý: {len(unique_words)}")
print(f"Danh sách một số từ duy nhất: {list(unique_words)[:100]}")  # Hiển thị 10 từ đầu tiên

Tổng số từ duy nhất trong X_data đã qua xử lý: 27973
Danh sách một số từ duy nhất: ['trích_dẫn', 'xuong', 'lan_truyền_thông_qua', 'manhđau', 'multi', 'gain', 'dặn', 'hệ_số', 'khum', 'tham_vấn', 'cafein', 'đấng', 'parry', 'cộng', 'bottle', 'súng_đạn', 'acetat', 'dently', 'bandha', 'si', 'verywellhealth', 'felodipine', 'doxepin', 'cay', 'kieng', 'wada', 'alginer', 'syndrome', 'thực_hiện_thời_điểm', 'xít', 'ii', 'như_hóa_trị', 'cấp_tính_cách', 'do_giảm', 'thực_hư', 'chả', 'evolving', 'răng_cửa', 'nảy', 'film', 'kiễng', 'creatinine', 'nôn_sốt', 'chuồn_chuồn', 'thường_trú', 'đánh_máy_chữ', 'ghiền', 'bọn', 'trầm_cảm_amitriptyline', 'quick', 'xư', 'terbutanyl', 'huy', 'axit', 'nghiền_ngẫm', 'lượngt', 'tia', 'tiêu_điểm_ảnh', 'ekip', 'ethionamide', 'bẽ_mặt', 'cherry', 'gió_máy_lạnh', 'cấy', 'inositol', 'lựcđỏ', 'wallace', 'radii', 'trưởng', 'khí_carbonic', 'tác_động_vật_lý', 'trầm_trọng_lượng', 'trường_hợp_dây', 'di_căn_hóa_trị', 'muscular', 'sôi_chú', 'roche', 'cá_bống', 'trí_tưởng_tượng', 'tự